In [1]:
import os
import pandas as pd
import geopandas as gpd
os.chdir(os.path.expanduser('~/TOP-Sprint/analysis'))
from src.features.count_jobs import count_jobs
from src.features.count_jobs import count_all_jobs
from src.features.jobs_vulnerability import get_worker_svi
from src.features.build_stop_features import get_transit_walksheds
from src.features.areal_interpolation import areal_interpolation
from src.features.count_jobs import count_jobs_to_subtract
import src.features.areal_interpolation as areal 
from src.features.build_stop_features import get_stops_features

In [2]:

BASE_WALKSHED_PATH = '/home/data/osm/'
CITY_DIRS = ['nyc','hampton_roads']

NYC_TRANSIT_WALKSHED = '/home/data/osm/nyc/walksheds/transit_walkshed_fixed.geojson'
HR_TRANSIT_WALKSHED = '/home/data/osm/hampton_roads/walksheds/transit_walkshed_fixed.geojson'

NYC_WALK_GRAPH = '/home/data/osm/nyc/NYC_walk_graph.gpickle'
HR_WALK_GRAPH = '/home/data/osm/hampton_roads/HR_walk_graph.gpickle'

NYC_LODES = '/home/data/census/nyc/LODES/ny_od_main_JT01_2019.csv'
HR_LODES = '/home/data/census/hampton_roads/LODES/va_od_main_JT01_2019.csv'

NYC_BLOCKS = '/home/data/census/nyc/geo/block_groups.geojson'
HR_BLOCKS = '/home/data/census/hampton_roads/geo/block_groups.geojson'

NYC_TRACTS = '/home/data/census/nyc/geo/tracts.geojson'
HR_TRACTS = '/home/data/census/hampton_roads/geo/tracts.geojson'

SVI_PATH = '/home/data/social_vulnerability_index/SVI2020_US.csv'

In [3]:
nyc_poly_fixed,hr_poly_fixed = get_transit_walksheds()
nyc_lodes = pd.read_csv(NYC_LODES)
hr_lodes = pd.read_csv(HR_LODES)
nyc_block_groups = gpd.read_file(NYC_BLOCKS)
hr_block_groups = gpd.read_file(HR_BLOCKS)

In [ ]:
nyc_jobs = count_jobs(blocks=nyc_blocks, polygons=nyc_poly_fixed, 
                          LODES=nyc_lodes, polygon_id_col='id', crs='epsg:2263')

In [ ]:
%%time
t = hr_poly_fixed.overlay(hr_blocks, keep_geom_type = False)

In [ ]:
%%time
t = hr_poly_fixed.overlay(hr_blocks, keep_geom_type = True)

In [ ]:
%%time
areal_interpolation(hr_blocks, hr_lodes, hr_poly_fixed, polygon_id_col = "id", drop_cols = ["intersection_weight"], crs = "epsg:2283").reset_index()

In [4]:
hr_jobs = count_jobs(blocks=hr_blocks,
                     polygons=hr_poly_fixed, 
                     LODES=hr_lodes, 
                     polygon_id_col='id',
                     crs='epsg:2283')

start time: 0
process geo: 13
count all jobs: 45
count jobs to subtract: 152
final merge: 0
CPU times: user 3min 29s, sys: 1.23 s, total: 3min 30s
Wall time: 3min 30s


In [4]:
hr_jobs = count_jobs(blocks=hr_block_groups,
                     polygons=hr_poly_fixed, 
                     LODES=hr_lodes, 
                     polygon_id_col='id',
                     crs='epsg:2283')

start time: 0
process geo: 7
count all jobs: 4
count jobs to subtract: 45
final merge: 0


In [4]:
hr_jobs = count_jobs(blocks=hr_block_groups,
                     polygons=hr_poly_fixed, 
                     LODES=hr_lodes, 
                     polygon_id_col='id',
                     crs='epsg:2283')

start time: 0
process geo: 7
count all jobs: 4
count jobs to subtract: 26
final merge: 0


In [5]:
nyc_jobs = count_jobs(blocks=nyc_block_groups,
                     polygons=nyc_poly_fixed, 
                     LODES=nyc_lodes, 
                     polygon_id_col='id',
                     crs='epsg:2263')

start time: 0
process geo: 22
count all jobs: 91
count jobs to subtract: 2537
final merge: 0


In [4]:
nyc_jobs = count_jobs(blocks=nyc_block_groups,
                     polygons=nyc_poly_fixed, 
                     LODES=nyc_lodes, 
                     polygon_id_col='id',
                     crs='epsg:2263')

start time: 0
process geo: 23
count all jobs: 92
adfa


MemoryError: Unable to allocate 200. GiB for an array with shape (26835654797,) and data type int64

In [ ]:
%%time
hr_jobs = count_all_jobs(blocks=hr_blocks,
                     polygons=hr_poly_fixed, 
                     LODES=hr_lodes, 
                     polygon_id_col='id',
                     crs='epsg:2283')

In [ ]:
%%time
hr_jobs = count_jobs_to_subtract(blocks=hr_blocks,
                     polygons=hr_poly_fixed, 
                     LODES=hr_lodes, 
                     polygon_id_col='id',
                     crs='epsg:2283')

In [9]:
 weights = areal.calculate_areal_weights(polygons = nyc_poly_fixed, 
                                         blocks = nyc_blocks,
                                         polygon_id_col = "id")

TypeError: calculate_areal_weights() got an unexpected keyword argument 'blocks'

In [10]:
%%time
crs = "epsg:2263"
polygons = nyc_poly_fixed.to_crs(crs)
blocks = nyc_blocks.to_crs(crs)
blocks = areal.calculate_census_areas(blocks)

LODES["w_geocode"] = LODES["w_geocode"].astype(str)
LODES["h_geocode"] = LODES["h_geocode"].astype(str)
LODES = LODES[["w_geocode", "h_geocode", "S000"]]

NameError: name 'LODES' is not defined

In [ ]:
%%time
jobs_full = count_all_jobs(blocks, polygons, LODES, polygon_id_col, crs)

In [22]:
LODES = nyc_lodes

In [23]:
LODES["w_geocode"] = LODES["w_geocode"].astype(str).str.slice(0,12)
LODES["h_geocode"] = LODES["h_geocode"].astype(str).str.slice(0,12)

In [28]:
LODES = LODES.groupby(["w_geocode", "h_geocode"]).agg(S000 = ("S000", "sum")).reset_index()

,w_geocode,h_geocode,S000
0,360010001001,360010001001,8
1,360010001001,360010001002,6
2,360010001001,360010002001,5
3,360010001001,360010002003,3
4,360010001001,360010002004,5
...,...,...,...
4301037,361231505004,361231504004,9
4301038,361231505004,361231505001,12
4301039,361231505004,361231505002,10
4301040,361231505004,361231505003,24


In [29]:
LODES

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,360010001001,360010002004,1,1,0,0,0,0,1,0,0,1,20211018
1,360010001001,360930320001,1,0,1,0,0,0,1,0,0,1,20211018
2,360010001001,360010002003,1,1,0,0,0,0,1,1,0,0,20211018
3,360010001001,360010007003,1,0,1,0,0,0,1,1,0,0,20211018
4,360010001001,360010021002,1,0,0,1,0,0,1,0,1,0,20211018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7184230,361231505004,361231505003,1,0,1,0,0,1,0,0,1,0,20211018
7184231,361231505004,361231505003,1,0,1,0,0,0,1,0,1,0,20211018
7184232,361231505004,361231505003,1,0,0,1,0,1,0,0,1,0,20211018
7184233,361231505004,360550113012,1,0,0,1,0,0,1,0,0,1,20211018


In [34]:
nyc_lodes

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,360010001001004,360010002004006,1,1,0,0,0,0,1,0,0,1,20211018
1,360010001001004,360930320001020,1,0,1,0,0,0,1,0,0,1,20211018
2,360010001001005,360010002003009,1,1,0,0,0,0,1,1,0,0,20211018
3,360010001001005,360010007003002,1,0,1,0,0,0,1,1,0,0,20211018
4,360010001001005,360010021002003,1,0,0,1,0,0,1,0,1,0,20211018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7184230,361231505004020,361231505003007,1,0,1,0,0,1,0,0,1,0,20211018
7184231,361231505004020,361231505003010,1,0,1,0,0,0,1,0,1,0,20211018
7184232,361231505004020,361231505003016,1,0,0,1,0,1,0,0,1,0,20211018
7184233,361231505004023,360550113012014,1,0,0,1,0,0,1,0,0,1,20211018


In [31]:
hr_lodes

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,510010901001000,510010901001018,1,0,0,1,0,1,0,0,0,1,20211018
1,510010901001000,510010901001049,2,1,0,1,0,2,0,0,0,2,20211018
2,510010901001000,510010901001055,1,0,0,1,1,0,0,0,0,1,20211018
3,510010901001000,510010901001065,1,0,1,0,0,1,0,0,0,1,20211018
4,510010901001000,510010901002001,2,0,1,1,0,0,2,0,0,2,20211018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750620,518400003024037,518400003012050,1,0,0,1,0,1,0,0,0,1,20211018
2750621,518400003024037,518400003021029,1,1,0,0,1,0,0,0,0,1,20211018
2750622,518400003024037,518400003022022,1,0,0,1,1,0,0,0,0,1,20211018
2750623,518400003024037,518400003023001,1,0,0,1,1,0,0,0,0,1,20211018


In [4]:
%%time
get_stops_features()

Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_bronx
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_brooklyn
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_queens
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_manhattan
Processing feed: /home/data/transit_feed_data/mta_feeds/google_transit_staten_island
Processing feed: /home/data/transit_feed_data/mta_feeds/mta_subway
Processing feed: /home/data/transit_feed_data/mta_feeds/bus_company
Processing feed: /home/data/transit_feed_data/hrt_feeds/hrt
Processing feed: /home/data/transit_feed_data/hrt_feeds/williamsburg
(19979, 12)
Adding First St. flood risk
(19979, 16)
Adding Hospital Access
(19979, 17)
Adding Number of jobs
Getting NYC jobs
start time: 0
process geo: 22
count all jobs: 92
count jobs to subtract: 2544
final merge: 0
Getting Hampton Roads jobs
start time: 0
process geo: 7
count all jobs: 4
count jobs to subtract: 44
final merge: 0


,stop_id,route_type,stop_name,stop_lat,stop_lon,feed_name,routes_serviced,routes_serviced_str,GEOID_2010,GEOID_2020,...,risk_category,access_to_hospital,jobs,jobs_cat,SVI_total,SVI_SES,SVI_household,SVI_race,SVI_housing_transport,worker_vulnerability_cat
0,404028,Bus,BROADWAY/W 219 ST,40.871953,-73.913137,google_transit_bronx,"M100, BX7, BX20","M100, BX7, BX20",36061029700,36061029700,...,0,2,5439.675745,1,9.868632,3.188302,2.828710,0.793351,3.057867,2
1,404028,Bus,BROADWAY/W 219 ST,40.871953,-73.913137,google_transit_bronx,"M100, BX7, BX20","M100, BX7, BX20",36061029700,36061029700,...,0,2,5439.675745,1,9.894621,3.202161,2.834499,0.795407,3.062126,2
2,404028,Bus,BROADWAY/W 219 ST,40.871953,-73.913137,google_transit_bronx,"M100, BX7, BX20","M100, BX7, BX20",36061029700,36061029700,...,0,2,5939.511285,1,9.868632,3.188302,2.828710,0.793351,3.057867,2
3,404028,Bus,BROADWAY/W 219 ST,40.871953,-73.913137,google_transit_bronx,"M100, BX7, BX20","M100, BX7, BX20",36061029700,36061029700,...,0,2,5939.511285,1,9.894621,3.202161,2.834499,0.795407,3.062126,2
4,404028,Bus,BROADWAY/W 219 ST,40.871770,-73.913094,bus_company,BXM1,BXM1,36061029900,36061029900,...,2,2,5439.675745,1,9.868632,3.188302,2.828710,0.793351,3.057867,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26160,2576874,Bus,Old Williamsburg Rd at Rising Sun OB,37.23274,-76.54599,williamsburg,15221,15221,51199050500,51199050500,...,1,0,6.429306,0,7.040955,2.212508,2.447829,0.494133,1.886486,0
26161,2576873,Bus,Old Williamsburg Rd at Woods of Yorktown,37.23425,-76.53737,williamsburg,15221,15221,51199050500,51199050500,...,1,0,4.272068,0,7.005895,2.197082,2.446495,0.487915,1.874404,0
26162,2576885,Bus,Barham Blvd at Rivermeade,37.22678,-76.51867,williamsburg,15221,15221,51199050500,51199050500,...,1,0,7.331028,0,7.003025,2.195819,2.446385,0.487406,1.873415,0
26163,2576884,Bus,Barham Blvd at Yorktown Square,37.22698,-76.52173,williamsburg,15221,15221,51199050500,51199050500,...,1,0,9.695693,0,7.003025,2.195819,2.446385,0.487406,1.873415,0
